In [1]:
import argparse
import collections

import wandb
import time
import numpy as np
import os
import cv2

from PIL import Image

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader


from models.retina.retinanet import model, coco_eval
from models.retina.analyze_regression_moe import analyze_regression
from models.retina.retinanet.dataloader import CocoDataset, CocoSubDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, \
    Normalizer
from models.moe_layer.soft_gating_networks import SimpleGate
from models.moe_layer.static_gating_networks import SingleWeightingGatingNetwork


In [2]:
class_file = '/home/lb4653/mixture-of-experts-thesis/data/coco/subset.names'
classes = [line.strip() for line in open(class_file, 'r')]

In [3]:
    # Create the model
    retinanet = model.resnet50(num_classes=len(classes), pretrained=False)
    retinanet.freeze_bn()

    # Freeze all layers
    for param in retinanet.parameters():
        param.requires_grad = False

    # Add MoE Regressor
    gate = SimpleGate(in_channels=256, 
                        num_experts=4,
                        top_k=2,
                        use_noise=True,
                        name='SimpleGate',
                        loss_fkt='kl_divergence',
                        w_aux_loss=0.5
                        )

    retinanet.regressionModel = model.RegressionModelMoE(num_features_in=256, num_experts=4, top_k=2, gating_network=gate)
    file_model = wandb.restore('retinanet_final.tar', run_path='lukas-struppek/RetinaNet/b1468is2')
    retinanet.load_state_dict(torch.load('retinanet_final.tar')['model_state_dict'])

    retinanet = retinanet.cuda()



In [4]:
dataset_val = CocoSubDataset('/home/lb4653/mixture-of-experts-thesis/data/coco', classes=classes, set_name='val2017',
                                  transform=transforms.Compose([Normalizer(), Resizer()]))

loading annotations into memory...
Done (t=0.37s)
creating index...
index created!


In [5]:
coco_eval_result_complete = coco_eval.evaluate_coco(dataset_val, retinanet)

4097/4098Loading and preparing results...
DONE (t=1.87s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=26.46s).
Accumulating evaluation results...
DONE (t=3.62s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.384
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.585
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.401
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.157
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.403
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.552
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.297
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.482
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.522
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | 

In [6]:
coco_eval_result_single_experts = []
singleGate = SingleWeightingGatingNetwork(in_channels=256, expert_index=0, num_experts=4, name='SingleWeightGatingNetwork')
retinanet.regressionModel.gate = singleGate
for i in range(4):
    retinanet.regressionModel.gate.expert_index = i
    retinanet.to('cuda:0')
    coco_eval_result_single_experts.append(coco_eval.evaluate_coco(dataset_val, retinanet)['coco_eval'].stats.copy())

4097/4098Loading and preparing results...
DONE (t=2.44s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=27.67s).
Accumulating evaluation results...
DONE (t=3.69s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.365
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.581
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.384
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.393
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.518
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.283
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.464
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.504
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | 

In [7]:
for i in range(4):
    print('mAP expert ', i, coco_eval_result_single_experts[i][0])
print()
for i in range(4):
    print('AP50 expert ', i, coco_eval_result_single_experts[i][1])

mAP expert  0 0.3650288967189132
mAP expert  1 0.37295279973813045
mAP expert  2 0.3559625303375254
mAP expert  3 0.36642777004196947

AP50 expert  0 0.5810489528640568
AP50 expert  1 0.5825522325630457
AP50 expert  2 0.5728500772451078
AP50 expert  3 0.5809641011165386


In [8]:
coco_eval_result_varying_k = []
retinanet.regressionModel.gate = gate
for i in range(1, 5):
    retinanet.regressionModel.gate.top_k = i
    retinanet.regressionModel.top_k = i
    retinanet.to('cuda:0')
    coco_eval_result_varying_k.append(coco_eval.evaluate_coco(dataset_val, retinanet)['coco_eval'].stats.copy())

4097/4098Loading and preparing results...
DONE (t=2.47s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=26.01s).
Accumulating evaluation results...
DONE (t=3.53s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.378
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.583
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.397
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.155
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.398
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.543
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.293
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.475
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.515
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | 

In [9]:
for i in range(4):
    print('mAP expert ', i, coco_eval_result_varying_k[i][0])
print()
for i in range(4):
    print('AP50 expert ', i, coco_eval_result_varying_k[i][1])

mAP expert  0 0.37823608019006605
mAP expert  1 0.38350053514101623
mAP expert  2 0.38342461181567233
mAP expert  3 0.3831239952232973

AP50 expert  0 0.5832738723984077
AP50 expert  1 0.58453078353178
AP50 expert  2 0.5857936813267917
AP50 expert  3 0.5853373356144652


In [10]:
coco_eval_result_varying_k

[array([0.37823608, 0.58327387, 0.3969403 , 0.15518295, 0.39800273,
        0.54320978, 0.29286036, 0.47547243, 0.51543793, 0.31331817,
        0.55187021, 0.6925858 ]),
 array([0.38350054, 0.58453078, 0.40120415, 0.15735548, 0.40303119,
        0.55164851, 0.29717311, 0.48203405, 0.52173075, 0.31960132,
        0.55698502, 0.70046215]),
 array([0.38342461, 0.58579368, 0.40146094, 0.15759327, 0.40413959,
        0.55051489, 0.29708235, 0.48179648, 0.52171246, 0.32018669,
        0.55802238, 0.69786756]),
 array([0.383124  , 0.58533734, 0.40163955, 0.15704226, 0.40271278,
        0.55092216, 0.29659983, 0.48180386, 0.52199982, 0.31875231,
        0.55859097, 0.69891212])]